In [1]:
data_dir = '/content/drive/MyDrive/Data_Sets/heads_tails_dataset'

In [2]:
import os
os.listdir(data_dir)

['train', 'test']

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [4]:
test_path = data_dir + '/test/'
train_path = data_dir + '/train/'

In [5]:
test_path

'/content/drive/MyDrive/Data_Sets/heads_tails_dataset/test/'

In [6]:
os.listdir(test_path)

['heads', 'tails']

In [7]:
dim1 = []
dim2 = []

for image_filename in os.listdir(train_path + 'heads'):
  img = imread(train_path + 'heads/' + image_filename)
  d1,d2,colors = img.shape
  dim1.append(d1)
  dim2.append(d2)

In [8]:
image_shape = (100,100,3)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
image_gen = ImageDataGenerator(rotation_range = 20, 
                               width_shift_range = 0.1, 
                               height_shift_range=0.1, 
                               rescale = 1/255, 
                               shear_range = 0.1,
                               zoom_range = 0.1,
                               horizontal_flip=True,
                               fill_mode = 'nearest')

In [11]:
image_gen.flow_from_directory(train_path)

Found 2040 images belonging to 2 classes.


In [12]:
image_gen.flow_from_directory(test_path)

Found 59 images belonging to 2 classes.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPool2D, Dense, Dropout, Conv2D,Flatten

In [14]:
model = Sequential()
model.add(Conv2D(filters = 64,kernel_size = (3,3), input_shape = (100,100,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128,kernel_size = (3,3), input_shape = (100,100,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
# This next layer should have filter number equal to number of classes
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',
              metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 67712)             0         
                                                                 
 dense (Dense)               (None, 256)               1

In [16]:
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
early_stop = EarlyStopping(monitor = 'val_loss',patience = 2)

In [18]:
batch_size = 32

In [19]:
train_image_generator = image_gen.flow_from_directory(train_path, target_size = (100,100),
                                                      color_mode = 'rgb',
                                                      batch_size = batch_size,
                                                      class_mode='binary',
                                                      shuffle = True)

Found 2040 images belonging to 2 classes.


In [20]:
test_image_generator = image_gen.flow_from_directory(test_path, target_size = (100,100),
                                                      color_mode = 'rgb',
                                                      batch_size = batch_size,
                                                      class_mode='binary',
                                                     shuffle = False)

Found 59 images belonging to 2 classes.


In [21]:
train_image_generator.class_indices

{'heads': 0, 'tails': 1}

In [22]:
test_image_generator.class_indices

{'heads': 0, 'tails': 1}

In [ ]:
results = model.fit_generator(train_image_generator, 
                              epochs = 100,
                              validation_data = test_image_generator,
                              steps_per_epoch = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/100
10/10 [==============================] - 53s 5s/step - loss: 1.1887 - accuracy: 0.5469 - val_loss: 0.6765 - val_accuracy: 0.6271
Epoch 2/100
10/10 [==============================] - 40s 4s/step - loss: 0.6634 - accuracy: 0.6156 - val_loss: 0.7235 - val_accuracy: 0.4576
Epoch 3/100
10/10 [==============================] - 28s 3s/step - loss: 0.5585 - accuracy: 0.6656 - val_loss: 0.8189 - val_accuracy: 0.5254
Epoch 4/100
10/10 [==============================] - 25s 2s/step - loss: 0.4977 - accuracy: 0.7656 - val_loss: 0.6627 - val_accuracy: 0.5932
Epoch 5/100
10/10 [==============================] - 25s 2s/step - loss: 0.4402 - accuracy: 0.7906 - val_loss: 0.4615 - val_accuracy: 0.7288
Epoch 6/100
10/10 [==============================] - 23s 2s/step - loss: 0.4131 - accuracy: 0.8156 - val_loss: 0.4813 - val_accuracy: 0.7458
Epoch 7/100
10/10 [==============================] - 16s 2s/step - loss: 0.4464 - accuracy: 0.8344 - val_loss: 0.4793 - val_accuracy: 0.7797
Epoch 8/100
1

In [ ]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://433887a5-3aab-40f3-a0c4-41489c595e43/assets


In [ ]:
pred = model.predict_generator(test_image_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [ ]:
predictions = pred > 0.5

In [ ]:
predictions

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [ True]])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(test_image_generator.classes, predictions))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        37
           1       1.00      0.77      0.87        22

    accuracy                           0.92        59
   macro avg       0.94      0.89      0.90        59
weighted avg       0.93      0.92      0.91        59



In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
my_image = image.load_img('/content/drive/MyDrive/data-coins/test/2 Rupees/Two_95.jpg',target_size = (200,200,3))

FileNotFoundError: ignored

In [ ]:
my_image_array = image.img_to_array(my_image)

In [ ]:
img_arr = np.expand_dims(my_image_array, axis = 0)

In [ ]:
model.predict(img_arr)